In [ ]:
%matplotlib inline

In [ ]:
import os
import pandas as pd
import dotenv
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as sta

#environment variables
dotenv.load_dotenv('spinco.env')

#project variables
datapath= '\\'+os.environ['DATAPATH']
testInfo=pd.read_csv(datapath+'\Pacientes_COGNITION_codigos.csv')
patientInfo=pd.read_csv(datapath+'\Pacientes_COGNITION_testCognitivos.csv')

### some useful views of the data

In [ ]:
patientInfo

In [ ]:
patientInfo.describe()

In [ ]:
testInfo

### load C3 channels in a dictionary

In [ ]:
samplerate=500
signalpaths=os.listdir(datapath+'\\Archivos canales')
signalpaths=[a for a in signalpaths if a.endswith(".mat") ]
signalpaths=[datapath+'\\Archivos canales\\'+a for a in signalpaths]

In [ ]:
signals=dict()

In [ ]:
#this is not needed at this point but is the data structure I want to follow
#in the case of multiple channels each object would be a nested dictionary with the channel names
""" for i in range(len(signalpaths)):
    key=signalpaths[i][-19:-13]
    signals[key]=sio.loadmat(signalpaths[i])['C3_analogico'].flatten() """

In [ ]:
""" print(signals.keys()) """

In [ ]:
""" timepoints=np.linspace(0,len(signals[key])/samplerate,len(signals[key]))
for signal in signals.values():
   plt.plot(timepoints,signal)
plt.legend(signals.keys(),loc='upper right')
plt.show() """

### load spindles annotations
this is what we actually need

In [ ]:
annopaths=os.listdir(datapath+'\\Labels')
annopaths=[a for a in annopaths if a.endswith(".csv") ]
annopaths=[datapath+'\\Labels\\'+a for a in annopaths]

In [ ]:
annopaths

read one

In [ ]:
annotations=pd.read_csv(annopaths[0])
annotations['Codigo']='COG001'

In [ ]:
annotations

iterate over all

In [ ]:
annotations=pd.DataFrame(columns=annotations.columns)
for i in range(len(annopaths)):
    aux=pd.read_csv(annopaths[i])
    #aux['Codigo'] = 'COG00'+str(i+1)   #<- it works but is risky
    aux['Codigo'] = annopaths[i][-21:-15]   #-< better to get it from the file name
    annotations=pd.concat([annotations,aux])

In [ ]:
annotations.columns

In [ ]:
annotations.groupby('Codigo').Duracion.describe()

In [ ]:
annotations.groupby(['Codigo','Fase']).Duracion.describe()

In [ ]:
aux=annotations
aux['count']=0
aux=aux.groupby(['Codigo','Fase']).count().reset_index()

In [ ]:
aux

In [ ]:
fig=px.bar(aux,x='Codigo',y='count',color='Fase',barmode='group')
fig.update_layout(title_text='#spindles for each patient, colored by sleep stage', title_x=0.5)
fig.show()

### correlations

In [ ]:
aux=annotations
# there's probably a better way to do this ---------------->
aux['spindleCount']=0
aux=aux.groupby(['Codigo']).count().reset_index().set_index('Codigo')
aux=aux[['spindleCount']]
# <---------------------------------------------------------
aux


In [ ]:
merged=aux.join(patientInfo.set_index('Codigo'),on='Codigo')

In [ ]:
merged

In [ ]:
r=merged.corr(method='spearman')

In [ ]:
from scipy.stats import spearmanr
def spearmanr_pval(x,y):
        return spearmanr(x,y)[1]

def spearmanr_r(x,y):
        return spearmanr(x,y)[0]

r_scipy=merged.corr(method=spearmanr_r)

In [ ]:
pval=merged.corr(method=spearmanr_pval)

In [ ]:
pval

In [ ]:
import seaborn as sns # For pairplots and heatmaps
plt.figure(figsize=(15,8))
heatmap = sns.heatmap(r, vmin=-1, vmax=1, annot=True)
plt.title("Spearman Correlation")

In [ ]:
# double check
plt.figure(figsize=(15,8))
heatmap = sns.heatmap(r, vmin=-1, vmax=1, annot=True)
plt.title("Spearman Correlation using scipy")

In [ ]:
plt.figure(figsize=(15,8))
heatmap = sns.heatmap(pval, vmin=-1, vmax=1, annot=True)
plt.title("Spearman Correlation p-values")

In [ ]:
merged['Codigo']=merged.index

In [ ]:
#px.scatter(merged,x='Peso',y='spindleCount',trendline='lowess',trendline_options=dict(frac=0.8), trendline_color_override="green")
px.scatter(merged,x='Peso',y='spindleCount',text='Codigo')

In [ ]:
#tripple check, pearson's correlations should be even worse
r_pear=merged.corr(method='pearson')
plt.figure(figsize=(15,8))
heatmap = sns.heatmap(r_pear, vmin=-1, vmax=1, annot=True)
plt.title("Pearson Correlation")

In [ ]:
plt.figure(figsize=(15,8))
heatmap = sns.heatmap(r, vmin=-1, vmax=1, annot=True)
plt.title("Spearman Correlation using scipy")